In [2]:
import os
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import lightgbmlss
import optuna
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from properscoring import crps_gaussian, crps_ensemble
import random
import gpytorch
import tqdm.auto as tqdm
from lightgbmlss.model import *
from lightgbmlss.distributions.Gaussian import *
from drf import drf
from pygam import LinearGAM, s, f
from utils import EarlyStopping, train, train_trans, train_no_early_stopping, train_trans_no_early_stopping, train_GP, ExactGPModel
from torch.utils.data import TensorDataset, DataLoader

SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task_id=361072

    

In [3]:
# Create the checkpoint directory if it doesn't exist
os.makedirs('CHECKPOINTS/CLUSTERING', exist_ok=True)
CHECKPOINT_PATH = f'CHECKPOINTS/CLUSTERING/task_{task_id}.pt'

print(f"Task {task_id}")

task = openml.tasks.get_task(task_id)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

# Set the random seed for reproducibility
N_TRIALS=2
N_SAMPLES=100
PATIENCE=40
N_EPOCHS=1000
GP_ITERATIONS=1000
BATCH_SIZE=1024
seed=10
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)


# New new implementation
N_CLUSTERS=20
# calculate the mean and covariance matrix of the dataset
mean = np.mean(X, axis=0)
cov = np.cov(X.T)
scaler = StandardScaler()

# transform data to compute the clusters
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init="auto").fit(X_scaled)
distances=[]
mahalanobis_dist=[]
counts=[]
ideal_len=len(kmeans.labels_)/5
for i in np.arange(N_CLUSTERS):
    distances.append(np.abs(np.sum(kmeans.labels_==i)-ideal_len))
    counts.append(np.sum(kmeans.labels_==i))
    mean_k= np.mean(X.loc[kmeans.labels_==i,:], axis=0)
    mahalanobis_dist.append(mahalanobis(mean_k, mean, np.linalg.inv(cov)))

dist_df=pd.DataFrame(data={'mahalanobis_dist': mahalanobis_dist, 'count': counts}, index=np.arange(N_CLUSTERS))
dist_df=dist_df.sort_values('mahalanobis_dist', ascending=False)
dist_df['cumulative_count']=dist_df['count'].cumsum()
dist_df['abs_diff']=np.abs(dist_df['cumulative_count']-ideal_len)

final=(np.where(dist_df['abs_diff']==np.min(dist_df['abs_diff']))[0])[0]
labelss=dist_df.index[0:final+1].to_list()
labels=pd.Series(kmeans.labels_).isin(labelss)
labels.index=X.index
close_index=labels.index[np.where(labels==False)[0]]
far_index=labels.index[np.where(labels==True)[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

# calculate the mean and covariance matrix of the dataset
mean_ = np.mean(X_train, axis=0)
cov_ = np.cov(X_train.T)
scaler_ = StandardScaler()

# transform data to compute the clusters
X_train_scaled = scaler_.fit_transform(X_train)

kmeans_ = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init="auto").fit(X_train_scaled)
distances_=[]
counts_=[]
mahalanobis_dist_=[]
ideal_len_=len(kmeans_.labels_)/5
for i in np.arange(N_CLUSTERS):
    distances_.append(np.abs(np.sum(kmeans_.labels_==i)-ideal_len_))
    counts_.append(np.sum(kmeans_.labels_==i))
    mean_k_= np.mean(X_train.loc[kmeans_.labels_==i,:], axis=0)
    mahalanobis_dist_.append(mahalanobis(mean_k_, mean_, np.linalg.inv(cov_)))

dist_df_=pd.DataFrame(data={'mahalanobis_dist': mahalanobis_dist_, 'count': counts_}, index=np.arange(N_CLUSTERS))
dist_df_=dist_df_.sort_values('mahalanobis_dist', ascending=False)
dist_df_['cumulative_count']=dist_df_['count'].cumsum()
dist_df_['abs_diff']=np.abs(dist_df_['cumulative_count']-ideal_len_)

final_=(np.where(dist_df_['abs_diff']==np.min(dist_df_['abs_diff']))[0])[0]
labelss_=dist_df_.index[0:final_+1].to_list()
labels_=pd.Series(kmeans_.labels_).isin(labelss_)
labels_.index=X_train.index
close_index_=labels_.index[np.where(labels_==False)[0]]
far_index_=labels_.index[np.where(labels_==True)[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]


# Convert data to PyTorch tensors
X_train__tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train__tensor = torch.tensor(y_train_.values, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert to use GPU if available
if torch.cuda.is_available():
    X_train__tensor = X_train__tensor.cuda()
    y_train__tensor = y_train__tensor.cuda()
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

# Create flattened versions of the data
y_val_np = y_val.values.flatten()
y_test_np = y_test.values.flatten()

# Create TensorDatasets for training and validation sets
train__dataset = TensorDataset(X_train__tensor, y_train__tensor)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for training and validation sets
train__loader = DataLoader(train__dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Task 361072


Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.


In [56]:
dtrain_ = lgb.Dataset(torch.tensor(X_train_.values, dtype=torch.float32).clone().detach(), label=y_train_.values)

def boosted(trial):

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.5, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 30),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
    }
    opt_params = params.copy()
    n_rounds = opt_params["n_estimators"]
    del opt_params["n_estimators"]
    opt_params['feature_pre_filter']=False

    # Use LightGBMLossGuideRegressor for distributional prediction
    boosted_tree_model = LightGBMLSS(Gaussian(stabilization="None", response_fn="exp", loss_fn="nll"))
    boosted_tree_model.train(opt_params, dtrain_, num_boost_round=n_rounds)

    # Predict both the mean and standard deviation
    pred_params=boosted_tree_model.predict(X_val, pred_type="parameters")
    y_val_hat_boost=pred_params['loc']
    y_val_hat_std = pred_params['scale']

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_boost[i], sig=y_val_hat_std[i]) for i in range(len(y_val))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_boost = optuna.samplers.TPESampler(seed=seed)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

np.random.seed(seed)
quantiles=list(np.random.uniform(0,1,N_SAMPLES))
def rf(trial):
    params = {'num_trees': trial.suggest_int('num_trees', 100, 500),
        'mtry': trial.suggest_int('mtry', 1, 30),
        'min_node_size': trial.suggest_int('min_node_size', 10, 100)}
    
    drf_model = drf(**params, seed=seed)
    drf_model.fit(X_train_, y_train_)
    
    # Generate a sample from the drf model for each data point
    y_val_hat=drf_model.predict(newdata = X_val, functional = "quantile", quantiles=quantiles)

    # Calculate the CRPS for each prediction
    crps_values = [crps_ensemble(y_val_np[i], y_val_hat.quantile[i].reshape(-1)) for i in range(len(y_val_np))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_drf = optuna.samplers.TPESampler(seed=seed)
study_drf = optuna.create_study(sampler=sampler_drf, direction='minimize')
study_drf.optimize(rf, n_trials=N_TRIALS)


def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01, log=True),
            'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
            'num_layer': trial.suggest_int('num_layer', 2, 5),
            'hidden_dim': trial.suggest_int('hidden_dim', 100, 500),
            'resblock': trial.suggest_categorical('resblock', [True, False])}
    params['noise_dim']=params['hidden_dim']

    # Check if CUDA is available and if so, move the tensors and the model to the GPU
    if torch.cuda.is_available():
        engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'], device="cuda")
    else:
        engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'])
    
    # Generate a sample from the engression model for each data point
    y_val_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_val.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_val))]

    # Calculate the CRPS for each prediction
    crps_values = [crps_ensemble(y_val_np[i], np.array(y_val_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_val_np))]

    return np.mean(crps_values)

sampler_engression = optuna.samplers.TPESampler(seed=seed)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)


dtrain = lgb.Dataset(torch.tensor(X_train.values, dtype=torch.float32).clone().detach(), label=y_train.values)
opt_params = study_boost.best_params.copy()
n_rounds = opt_params["n_estimators"]
del opt_params["n_estimators"]
opt_params['feature_pre_filter']=False
# Use LightGBMLossGuideRegressor for distributional prediction
boosted_tree_model = LightGBMLSS(Gaussian(stabilization="None", response_fn="exp", loss_fn="nll"))
boosted_tree_model.train(opt_params, dtrain, num_boost_round=n_rounds)
# Predict both the mean and standard deviation
pred_params=boosted_tree_model.predict(X_test, pred_type="parameters")
y_test_hat_boost=pred_params['loc']
y_test_hat_std = pred_params['scale']
# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_boost[i], sig=y_test_hat_std[i]) for i in range(len(y_test))]
# Return the mean CRPS as the objective to be minimized
CRPS_boosted=np.mean(crps_values)

drf_model=drf(**study_drf.best_params, seed=seed)
drf_model.fit(X_train, y_train)
# Generate a sample from the drf model for each data point
y_test_hat_drf=drf_model.predict(newdata = X_test, functional = "quantile", quantiles=quantiles)
# Calculate the CRPS for each prediction
crps_values = [crps_ensemble(y_test_np[i], y_test_hat_drf.quantile[i].reshape(-1)) for i in range(len(y_test_np))]
# Return the mean CRPS as the objective to be minimized
CRPS_rf=np.mean(crps_values)

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
y_test_hat_linreg=lin_reg.predict(X_test)
# Calculate the standard deviation of the residuals
std_dev = np.std(y_test - y_test_hat_linreg)
# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_linreg[i], sig=std_dev) for i in range(len(y_test_np))]
CRPS_linreg = np.mean(crps_values)

params=study_engression.best_params
params['noise_dim']=params['hidden_dim']
X_train_tensor = torch.Tensor(np.array(X_train))
y_train_tensor = torch.Tensor(np.array(y_train).reshape(-1,1))

# Check if CUDA is available and if so, move the tensors and the model to the GPU
if torch.cuda.is_available():
    engressor_model=engression(X_train_tensor, y_train_tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'], device="cuda")
else:
    engressor_model=engression(X_train_tensor, y_train_tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'])
# Generate a sample from the engression model for each data point
y_test_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_test.values[i]])).cuda() if torch.cuda.is_available() else torch.Tensor(np.array([X_test.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_test))]
# Calculate the CRPS for each prediction
crps_values = [crps_ensemble(y_test_np[i], np.array(y_test_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_test_np))]
CRPS_engression=np.mean(crps_values)

print("CRPS linear regression: ",CRPS_linreg)
print("CRPS boosted trees", CRPS_boosted)
print("CRPS random forest", CRPS_rf)
print("CRPS engression", CRPS_engression)

[I 2024-03-04 19:02:45,213] A new study created in memory with name: no-name-356d681b-1170-40ba-b0c6-b3f0b278d480
[I 2024-03-04 19:02:52,404] Trial 0 finished with value: 6.937608925970823 and parameters: {'learning_rate': 0.0713003929222653, 'n_estimators': 108, 'reg_lambda': 0.005044685709888605, 'max_depth': 23, 'min_child_samples': 55}. Best is trial 0 with value: 6.937608925970823.
[I 2024-03-04 19:02:55,046] Trial 1 finished with value: 12.527109038809398 and parameters: {'learning_rate': 0.0006784471913345375, 'n_estimators': 179, 'reg_lambda': 0.0699481785242808, 'max_depth': 6, 'min_child_samples': 18}. Best is trial 0 with value: 6.937608925970823.
[I 2024-03-04 19:02:55,049] A new study created in memory with name: no-name-ea90304b-f111-4642-bd43-36333a33cebc


[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

[I 2024-03-04 19:03:00,524] Trial 0 finished with value: 9.07003089184482 and parameters: {'num_trees': 409, 'mtry': 1, 'min_node_size': 67}. Best is trial 0 with value: 9.07003089184482.


[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

[I 2024-03-04 19:03:08,247] Trial 1 finished with value: 7.275542030588492 and parameters: {'num_trees': 400, 'mtry': 15, 'min_node_size': 30}. Best is trial 1 with value: 7.275542030588492.
[I 2024-03-04 19:03:08,250] A new study created in memory with name: no-name-7a4ea8ea-e886-40fc-89d9-9a08012593dc


Running on CPU.

Residual blocks (skip-connections) are typically recommended for more than 2 layers; turn it on by setting resblock=True.
Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1024.
[Epoch 1 (0%), batch 6] energy-loss: 0.4376,  E(|Y-Yhat|): 1.0720,  E(|Yhat-Yhat'|): 1.2687
[Epoch 100 (84%), batch 6] energy-loss: 0.2423,  E(|Y-Yhat|): 0.4711,  E(|Yhat-Yhat'|): 0.4575

Training loss on the original (non-standardized) scale:
	Energy-loss: 1.1010,  E(|Y-Yhat|): 2.3519,  E(|Yhat-Yhat'|): 2.5017

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 


[I 2024-03-04 19:04:49,599] Trial 0 finished with value: 3.041407315526319 and parameters: {'learning_rate': 0.0034885205571560775, 'num_epoches': 118, 'num_layer': 4, 'hidden_dim': 400, 'resblock': True}. Best is trial 0 with value: 3.041407315526319.


Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1024.
[Epoch 1 (0%), batch 6] energy-loss: 0.6360,  E(|Y-Yhat|): 0.8290,  E(|Yhat-Yhat'|): 0.3860
[Epoch 100 (13%), batch 6] energy-loss: 0.2307,  E(|Y-Yhat|): 0.4422,  E(|Yhat-Yhat'|): 0.4230
[Epoch 200 (25%), batch 6] energy-loss: 0.2261,  E(|Y-Yhat|): 0.4416,  E(|Yhat-Yhat'|): 0.4311
[Epoch 300 (38%), batch 6] energy-loss: 0.2274,  E(|Y-Yhat|): 0.4418,  E(|Yhat-Yhat'|): 0.4289
[Epoch 400 (51%), batch 6] energy-loss: 0.2288,  E(|Y-Yhat|): 0.4356,  E(|Yhat-Yhat'|): 0.4136
[Epoch 500 (64%), batch 6] energy-loss: 0.2395,  E(|Y-Yhat|): 0.4487,  E(|Yhat-Yhat'|): 0.4183
[Epoch 600 (76%), batch 6] energy-loss: 0.2258,  E(|Y-Yhat|): 0.4368,  E(|Yhat-Yhat'|): 0.4220
[Epoch 700 (89%), batch 6] energy-loss: 0.

[I 2024-03-04 19:06:17,801] Trial 1 finished with value: 4.233781359117593 and parameters: {'learning_rate': 0.0002489577954043506, 'num_epoches': 785, 'num_layer': 2, 'hidden_dim': 135, 'resblock': False}. Best is trial 0 with value: 3.041407315526319.


[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

In [57]:
dtrain_ = lgb.Dataset(torch.tensor(X_train_.values, dtype=torch.float32).clone().detach(), label=y_train_.values)

def boosted(trial):

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.5, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 30),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
    }
    opt_params = params.copy()
    n_rounds = opt_params["n_estimators"]
    del opt_params["n_estimators"]
    opt_params['feature_pre_filter']=False

    # Use LightGBMLossGuideRegressor for distributional prediction
    boosted_tree_model = LightGBMLSS(Gaussian(stabilization="None", response_fn="exp", loss_fn="nll"))
    boosted_tree_model.train(opt_params, dtrain_, num_boost_round=n_rounds)

    # Predict both the mean and standard deviation
    pred_params=boosted_tree_model.predict(X_val, pred_type="parameters")
    y_val_hat_boost=pred_params['loc']
    y_val_hat_std = pred_params['scale']

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_boost[i], sig=y_val_hat_std[i]) for i in range(len(y_val))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_boost = optuna.samplers.TPESampler(seed=seed)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

np.random.seed(seed)
quantiles=list(np.random.uniform(0,1,N_SAMPLES))
def rf(trial):
    params = {'num_trees': trial.suggest_int('num_trees', 100, 500),
        'mtry': trial.suggest_int('mtry', 1, 30),
        'min_node_size': trial.suggest_int('min_node_size', 10, 100)}
    
    drf_model = drf(**params, seed=seed)
    drf_model.fit(X_train_, y_train_)
    
    # Generate a sample from the drf model for each data point
    y_val_hat=drf_model.predict(newdata = X_val, functional = "quantile", quantiles=quantiles)

    # Calculate the CRPS for each prediction
    crps_values = [crps_ensemble(y_val_np[i], y_val_hat.quantile[i].reshape(-1)) for i in range(len(y_val_np))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_drf = optuna.samplers.TPESampler(seed=seed)
study_drf = optuna.create_study(sampler=sampler_drf, direction='minimize')
study_drf.optimize(rf, n_trials=N_TRIALS)


def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01, log=True),
            'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
            'num_layer': trial.suggest_int('num_layer', 2, 5),
            'hidden_dim': trial.suggest_int('hidden_dim', 100, 500),
            'resblock': trial.suggest_categorical('resblock', [True, False])}
    params['noise_dim']=params['hidden_dim']

    # Check if CUDA is available and if so, move the tensors and the model to the GPU
    if torch.cuda.is_available():
        engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'], device="cuda")
    else:
        engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'])
    
    # Generate a sample from the engression model for each data point
    y_val_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_val.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_val))]

    # Calculate the CRPS for each prediction
    crps_values = [crps_ensemble(y_val_np[i], np.array(y_val_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_val_np))]

    return np.mean(crps_values)

sampler_engression = optuna.samplers.TPESampler(seed=seed)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)


dtrain = lgb.Dataset(torch.tensor(X_train.values, dtype=torch.float32).clone().detach(), label=y_train.values)
opt_params = study_boost.best_params.copy()
n_rounds = opt_params["n_estimators"]
del opt_params["n_estimators"]
opt_params['feature_pre_filter']=False
# Use LightGBMLossGuideRegressor for distributional prediction
boosted_tree_model = LightGBMLSS(Gaussian(stabilization="None", response_fn="exp", loss_fn="nll"))
boosted_tree_model.train(opt_params, dtrain, num_boost_round=n_rounds)
# Predict both the mean and standard deviation
pred_params=boosted_tree_model.predict(X_test, pred_type="parameters")
y_test_hat_boost=pred_params['loc']
y_test_hat_std = pred_params['scale']
# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_boost[i], sig=y_test_hat_std[i]) for i in range(len(y_test))]
# Return the mean CRPS as the objective to be minimized
CRPS_boosted=np.mean(crps_values)

drf_model=drf(**study_drf.best_params, seed=seed)
drf_model.fit(X_train, y_train)
# Generate a sample from the drf model for each data point
y_test_hat_drf=drf_model.predict(newdata = X_test, functional = "quantile", quantiles=quantiles)
# Calculate the CRPS for each prediction
crps_values = [crps_ensemble(y_test_np[i], y_test_hat_drf.quantile[i].reshape(-1)) for i in range(len(y_test_np))]
# Return the mean CRPS as the objective to be minimized
CRPS_rf=np.mean(crps_values)

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
y_test_hat_linreg=lin_reg.predict(X_test)
# Calculate the standard deviation of the residuals
std_dev = np.std(y_test - y_test_hat_linreg)
# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_linreg[i], sig=std_dev) for i in range(len(y_test_np))]
CRPS_linreg = np.mean(crps_values)

params=study_engression.best_params
params['noise_dim']=params['hidden_dim']
X_train_tensor = torch.Tensor(np.array(X_train))
y_train_tensor = torch.Tensor(np.array(y_train).reshape(-1,1))

# Check if CUDA is available and if so, move the tensors and the model to the GPU
if torch.cuda.is_available():
    engressor_model=engression(X_train_tensor, y_train_tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'], device="cuda")
else:
    engressor_model=engression(X_train_tensor, y_train_tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'])
# Generate a sample from the engression model for each data point
y_test_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_test.values[i]])).cuda() if torch.cuda.is_available() else torch.Tensor(np.array([X_test.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_test))]
# Calculate the CRPS for each prediction
crps_values = [crps_ensemble(y_test_np[i], np.array(y_test_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_test_np))]
CRPS_engression=np.mean(crps_values)

print("CRPS linear regression: ",CRPS_linreg)
print("CRPS boosted trees", CRPS_boosted)
print("CRPS random forest", CRPS_rf)
print("CRPS engression", CRPS_engression)

[I 2024-03-04 19:08:30,657] A new study created in memory with name: no-name-e2a9d29b-e253-4507-af13-0e3a103bf9bd
[I 2024-03-04 19:08:33,473] Trial 0 finished with value: 6.937608925970823 and parameters: {'learning_rate': 0.0713003929222653, 'n_estimators': 108, 'reg_lambda': 0.005044685709888605, 'max_depth': 23, 'min_child_samples': 55}. Best is trial 0 with value: 6.937608925970823.
[I 2024-03-04 19:08:36,708] Trial 1 finished with value: 12.527109038809398 and parameters: {'learning_rate': 0.0006784471913345375, 'n_estimators': 179, 'reg_lambda': 0.0699481785242808, 'max_depth': 6, 'min_child_samples': 18}. Best is trial 0 with value: 6.937608925970823.
[I 2024-03-04 19:08:36,711] A new study created in memory with name: no-name-4f377d26-6161-4e4f-8457-c817e189b3ef


[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

[I 2024-03-04 19:08:47,006] Trial 0 finished with value: 9.07003089184482 and parameters: {'num_trees': 409, 'mtry': 1, 'min_node_size': 67}. Best is trial 0 with value: 9.07003089184482.


[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

[I 2024-03-04 19:08:59,240] Trial 1 finished with value: 7.275542030588492 and parameters: {'num_trees': 400, 'mtry': 15, 'min_node_size': 30}. Best is trial 1 with value: 7.275542030588492.
[I 2024-03-04 19:08:59,270] A new study created in memory with name: no-name-53c4582e-504f-440f-9a36-d447215e770f


Running on CPU.

Residual blocks (skip-connections) are typically recommended for more than 2 layers; turn it on by setting resblock=True.
Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1024.
[Epoch 1 (0%), batch 6] energy-loss: 0.4376,  E(|Y-Yhat|): 1.0720,  E(|Yhat-Yhat'|): 1.2687
[Epoch 100 (84%), batch 6] energy-loss: 0.2423,  E(|Y-Yhat|): 0.4711,  E(|Yhat-Yhat'|): 0.4575

Training loss on the original (non-standardized) scale:
	Energy-loss: 1.1010,  E(|Y-Yhat|): 2.3519,  E(|Yhat-Yhat'|): 2.5017

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 


[I 2024-03-04 19:11:02,423] Trial 0 finished with value: 3.041407315526319 and parameters: {'learning_rate': 0.0034885205571560775, 'num_epoches': 118, 'num_layer': 4, 'hidden_dim': 400, 'resblock': True}. Best is trial 0 with value: 3.041407315526319.


Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1024.
[Epoch 1 (0%), batch 6] energy-loss: 0.6360,  E(|Y-Yhat|): 0.8290,  E(|Yhat-Yhat'|): 0.3860
[Epoch 100 (13%), batch 6] energy-loss: 0.2307,  E(|Y-Yhat|): 0.4422,  E(|Yhat-Yhat'|): 0.4230
[Epoch 200 (25%), batch 6] energy-loss: 0.2261,  E(|Y-Yhat|): 0.4416,  E(|Yhat-Yhat'|): 0.4311
[Epoch 300 (38%), batch 6] energy-loss: 0.2274,  E(|Y-Yhat|): 0.4418,  E(|Yhat-Yhat'|): 0.4289
[Epoch 400 (51%), batch 6] energy-loss: 0.2288,  E(|Y-Yhat|): 0.4356,  E(|Yhat-Yhat'|): 0.4136
[Epoch 500 (64%), batch 6] energy-loss: 0.2395,  E(|Y-Yhat|): 0.4487,  E(|Yhat-Yhat'|): 0.4183
[Epoch 600 (76%), batch 6] energy-loss: 0.2258,  E(|Y-Yhat|): 0.4368,  E(|Yhat-Yhat'|): 0.4220
[Epoch 700 (89%), batch 6] energy-loss: 0.

[I 2024-03-04 19:13:02,384] Trial 1 finished with value: 4.233781359117593 and parameters: {'learning_rate': 0.0002489577954043506, 'num_epoches': 785, 'num_layer': 2, 'hidden_dim': 135, 'resblock': False}. Best is trial 0 with value: 3.041407315526319.


[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

In [4]:
y_val_tensor.flatten()

tensor([85., 81., 87.,  ..., 85., 85., 87.])

In [5]:
def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01, log=True),
            'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
            'num_layer': trial.suggest_int('num_layer', 2, 5),
            'hidden_dim': trial.suggest_int('hidden_dim', 100, 500),
            'resblock': trial.suggest_categorical('resblock', [True, False])}
    params['noise_dim']=params['hidden_dim']

    # Check if CUDA is available and if so, move the tensors and the model to the GPU
    if torch.cuda.is_available():
        engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'], device="cuda")
    else:
        engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'])
    
    # Generate a sample from the engression model for each data point
    y_val_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_val.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_val))]

    # Calculate the CRPS for each prediction
    crps_values = [crps_ensemble(y_val_np[i], np.array(y_val_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_val_np))]

    return np.mean(crps_values)

sampler_engression = optuna.samplers.TPESampler(seed=seed)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)


# Check if CUDA is available and if so, move the tensors and the model to the GPU
if torch.cuda.is_available():
    engressor_model=engression(X_train_tensor, y_train_tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'], device="cuda")
else:
    engressor_model=engression(X_train_tensor, y_train_tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'])
# Generate a sample from the engression model for each data point
y_test_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_test.values[i]])).cuda() if torch.cuda.is_available() else torch.Tensor(np.array([X_test.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_test))]
# Calculate the CRPS for each prediction
crps_values = [crps_ensemble(y_test_np[i], np.array(y_test_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_test_np))]
CRPS_engression=np.mean(crps_values)

print("CRPS linear regression: ",CRPS_linreg)
print("CRPS boosted trees", CRPS_boosted)
print("CRPS random forest", CRPS_rf)
print("CRPS engression", CRPS_engression)

[I 2024-03-05 10:01:42,647] A new study created in memory with name: no-name-e547f643-08e0-47ef-9c4c-a9171ff6ac54


Running on CPU.

Residual blocks (skip-connections) are typically recommended for more than 2 layers; turn it on by setting resblock=True.
Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1024.
[Epoch 1 (0%), batch 6] energy-loss: 0.4338,  E(|Y-Yhat|): 1.0227,  E(|Yhat-Yhat'|): 1.1779
[Epoch 100 (84%), batch 6] energy-loss: 0.2444,  E(|Y-Yhat|): 0.4991,  E(|Yhat-Yhat'|): 0.5094

Training loss on the original (non-standardized) scale:
	Energy-loss: 1.1617,  E(|Y-Yhat|): 2.4525,  E(|Yhat-Yhat'|): 2.5816

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a mismatch 


[I 2024-03-05 10:03:14,305] Trial 0 finished with value: 3.997297018729463 and parameters: {'learning_rate': 0.0034885205571560775, 'num_epoches': 118, 'num_layer': 4, 'hidden_dim': 400, 'resblock': True}. Best is trial 0 with value: 3.997297018729463.


Running on CPU.

Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1024.
[Epoch 1 (0%), batch 6] energy-loss: 0.5307,  E(|Y-Yhat|): 0.7151,  E(|Yhat-Yhat'|): 0.3689
[Epoch 100 (13%), batch 6] energy-loss: 0.2189,  E(|Y-Yhat|): 0.4321,  E(|Yhat-Yhat'|): 0.4265


[W 2024-03-05 10:03:29,467] Trial 1 failed with parameters: {'learning_rate': 0.0002489577954043506, 'num_epoches': 785, 'num_layer': 2, 'hidden_dim': 135, 'resblock': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_6472\2127494930.py", line 14, in engressor_NN
    engressor_model=engression(X_train__tensor, y_train__tensor.reshape(-1,1), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=BATCH_SIZE, resblock=params['resblock'])
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\engression\engression.py", line 47, in engression
    engressor.train(x, y, num_epoches=num_epoches, batch_size=bat

KeyboardInterrupt: 

In [6]:
import os
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import lightgbmlss
import optuna
from scipy.spatial.distance import mahalanobis
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from properscoring import crps_gaussian, crps_ensemble
import random
import gpytorch
import tqdm.auto as tqdm
from lightgbmlss.model import *
from lightgbmlss.distributions.Gaussian import *
from pygam import LinearGAM, s, f
from utils import EarlyStopping, train, train_trans, train_no_early_stopping, train_trans_no_early_stopping, train_GP, ExactGPModel
from torch.utils.data import TensorDataset, DataLoader
from drf import drf

SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task_id=361074

In [17]:
len(benchmark_suite.tasks)

19

In [7]:
# Create the checkpoint directory if it doesn't exist
os.makedirs('CHECKPOINTS/MAHALANOBIS', exist_ok=True)
CHECKPOINT_PATH = f'CHECKPOINTS/MAHALANOBIS/task_{task_id}.pt'

print(f"Task {task_id}")

task = openml.tasks.get_task(task_id)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

# Set the random seed for reproducibility
N_TRIALS=100
N_SAMPLES=100
PATIENCE=40
N_EPOCHS=1000
GP_ITERATIONS=1000
BATCH_SIZE=1024
seed=10
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)


# calculate the mean and covariance matrix of the dataset
mean = np.mean(X, axis=0)
cov = np.cov(X.T)

# calculate the Mahalanobis distance for each data point
mahalanobis_dist = [mahalanobis(x, mean, np.linalg.inv(cov)) for x in X.values]

mahalanobis_dist=pd.Series(mahalanobis_dist,index=X.index)
far_index=mahalanobis_dist.index[np.where(mahalanobis_dist>=np.quantile(mahalanobis_dist,0.8))[0]]
close_index=mahalanobis_dist.index[np.where(mahalanobis_dist<np.quantile(mahalanobis_dist,0.8))[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

mean = np.mean(X_train, axis=0)
cov = np.cov(X_train.T)

# calculate the Mahalanobis distance for each data point
mahalanobis_dist_ = [mahalanobis(x, mean, np.linalg.inv(cov)) for x in X_train.values]

mahalanobis_dist_=pd.Series(mahalanobis_dist_,index=X_train.index)
far_index_=mahalanobis_dist_.index[np.where(mahalanobis_dist_>=np.quantile(mahalanobis_dist_,0.8))[0]]
close_index_=mahalanobis_dist_.index[np.where(mahalanobis_dist_<np.quantile(mahalanobis_dist_,0.8))[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]


# Convert data to PyTorch tensors
X_train__tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train__tensor = torch.tensor(y_train_.values, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert to use GPU if available
if torch.cuda.is_available():
    X_train__tensor = X_train__tensor.cuda()
    y_train__tensor = y_train__tensor.cuda()
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

# Create flattened versions of the data
y_val_np = y_val.values.flatten()
y_test_np = y_test.values.flatten()

# Create TensorDatasets for training and validation sets
train__dataset = TensorDataset(X_train__tensor, y_train__tensor)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for training and validation sets
train__loader = DataLoader(train__dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Task 361074


Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.


LinAlgError: Singular matrix

In [8]:
X.shape

(16599, 16)

In [9]:
cov.shape

(16, 16)

In [10]:
np.linalg.inv(cov)

LinAlgError: Singular matrix

In [11]:
cov

array([[ 7.21391060e+04, -1.91769834e+02,  1.01992189e+01,
        -6.20669455e+00, -4.06397490e+01, -2.38630219e+01,
         7.67521688e+02,  7.20824919e-01, -2.46737556e+01,
         1.17628657e-02,  1.22922006e-02,  1.22922006e-02,
         1.24216423e-02, -8.98351803e-04, -1.40631499e-04,
         1.24216423e-02],
       [-1.91769834e+02,  5.87296133e+02,  5.15133831e-01,
         9.07528413e-01,  5.10515565e-01,  7.26653674e+00,
        -1.05121549e+02, -1.36988171e-02, -1.96439028e-01,
         4.29649933e-04,  4.58733734e-04,  4.58733734e-04,
         4.74203922e-04, -3.41005033e-05, -1.72670663e-05,
         4.74203922e-04],
       [ 1.01992189e+01,  5.15133831e-01,  7.73643481e-02,
         3.00655351e-03,  1.20565177e-03,  1.36462404e-02,
        -2.73906768e-01,  5.96023040e-04, -1.70299365e-02,
         1.27764791e-05,  1.67190809e-05,  1.67190809e-05,
         1.78566964e-05, -5.95066117e-06, -1.55978696e-06,
         1.78566964e-05],
       [-6.20669455e+00,  9.07528413e

In [13]:
np.linalg.det(cov)

0.0

In [16]:
from mahalanobis import Mahalanobis
Mahalanobis(X,X)

ERROR:MahalanobisND:Wrong format in calib_entries argument. Must be convertible to integer, list or np.array


ValueError: Wrong format in calib_entries argument. Must be convertible to integer, list or np.array

In [18]:
#### GAM model
def gam_model(trial):

    # Define the hyperparameters to optimize
    params = {'n_splines': trial.suggest_int('n_splines', 5, 20),
            'lam': trial.suggest_float('lam', 1e-3, 1, log=True)}

    # Create and train the model
    gam = LinearGAM(s(0, n_splines=params['n_splines'], lam=params['lam'])).fit(X_train_, y_train_)

    # Predict on the validation set and calculate the CRPS
    y_val_hat_gam = gam.predict(X_val)
    std_dev_error = np.std(y_val - y_val_hat_gam)
    crps_gam = [crps_gaussian(y_val_np[i], mu=y_val_hat_gam[i], sig=std_dev_error) for i in range(len(y_val_hat_gam))]
    crps_gam = np.mean(crps_gam)

    return crps_gam

# Create the sampler and study
sampler_gam = optuna.samplers.TPESampler(seed=seed)
study_gam = optuna.create_study(sampler=sampler_gam, direction='minimize')

# Optimize the model
study_gam.optimize(gam_model, n_trials=N_TRIALS)

# Create the final model with the best parameters
best_params = study_gam.best_params
final_gam_model = LinearGAM(s(0, n_splines=best_params['n_splines'], lam=best_params['lam']))

# Fit the model
final_gam_model.fit(X_train, y_train)

# Predict on the test set
y_test_hat_gam = final_gam_model.predict(X_test)

# Calculate the CRPS
std_dev_error = np.std(y_test - y_test_hat_gam)
crps_gam = [crps_gaussian(y_test_np[i], mu=y_test_hat_gam[i], sig=std_dev_error) for i in range(len(y_test_hat_gam))]
crps_gam = np.mean(crps_gam)
print("CRPS GAM: ", crps_gam)


[I 2024-03-05 17:27:38,141] A new study created in memory with name: no-name-21501702-7a56-4a74-98ed-f8366ed6a79e
[I 2024-03-05 17:27:38,755] Trial 0 finished with value: 22.947252239814784 and parameters: {'n_splines': 17, 'lam': 0.001154132971137168}. Best is trial 0 with value: 22.947252239814784.
[I 2024-03-05 17:27:38,823] Trial 1 finished with value: 10.740112483235158 and parameters: {'n_splines': 15, 'lam': 0.17636469336159113}. Best is trial 1 with value: 10.740112483235158.
[I 2024-03-05 17:27:38,875] Trial 2 finished with value: 10.896560638127578 and parameters: {'n_splines': 12, 'lam': 0.00472487079152679}. Best is trial 1 with value: 10.740112483235158.
[I 2024-03-05 17:27:38,936] Trial 3 finished with value: 10.703338919026589 and parameters: {'n_splines': 8, 'lam': 0.19124590142517375}. Best is trial 3 with value: 10.703338919026589.
[I 2024-03-05 17:27:38,990] Trial 4 finished with value: 11.200486892602623 and parameters: {'n_splines': 7, 'lam': 0.0018408544111075849}

IndexError: index 1586 is out of bounds for axis 0 with size 1586

In [2]:
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LogisticRegression 
import lightgbm as lgbm
import optuna
from scipy.spatial.distance import mahalanobis
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
import random
import gpytorch
import tqdm.auto as tqdm
import os
from pygam import LogisticGAM, s
import torch
from torch import nn
from torch.optim import Adam
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder 
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
from gpytorch.mlls.variational_elbo import VariationalELBO
from utils import EarlyStopping, train, train_trans, train_no_early_stopping, train_trans_no_early_stopping, train_GP, ExactGPModel
from torch.utils.data import TensorDataset, DataLoader

#SUITE_ID = 336 # Regression on numerical features
SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite
print(len(benchmark_suite.tasks))

16


In [12]:
X.columns

Index(['temp', 'windspeed_max', 'windspeed_avg', 'precipitation', 'dew_point',
       'humidity', 'hour', 'dayminute'],
      dtype='object')

In [3]:
SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite
print(len(benchmark_suite.tasks))

for task_id in benchmark_suite.tasks:
    print(task_id)
    task = openml.tasks.get_task(task_id)  # download the OpenML task
    dataset = task.get_dataset()

    X, y, categorical_indicator, attribute_names = dataset.get_data(
            dataset_format="dataframe", target=dataset.default_target_attribute)
    print(X.shape)
    for col in X.columns:
        if X[col].nunique() < 10:
            print(col, X[col].nunique())

19
361072


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


(8192, 21)
361073
(15000, 26)
361074
(16599, 16)
361076
(6497, 11)
361077
(13750, 33)
361078


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

(20640, 8)
361079
(22784, 16)
361080
(53940, 6)
361081
(10692, 8)
361082


...

(163065, 3)
361087
(13932, 13)
361088
(21263, 79)
361279


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

(8885, 42)
361280
(4177, 7)
361281


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

(5465575, 8)


In [13]:
#SUITE_ID = 336 # Regression on numerical features
SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite
print(len(benchmark_suite.tasks))

for task_id in benchmark_suite.tasks:
    print(task_id)
    task = openml.tasks.get_task(task_id)  # download the OpenML task
    dataset = task.get_dataset()

    X, y, categorical_indicator, attribute_names = dataset.get_data(
            dataset_format="dataframe", target=dataset.default_target_attribute)
    print(X.shape)
    for col in X.columns:
        if X[col].nunique() < 10:
            print(col, X[col].nunique())

361055


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


(16714, 10)
NumberOfDependents 9
361060
(38474, 7)
361061


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

(566602, 10)
361062


...

(13272, 20)
361276


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


(3434, 419)
D337 9
D350 8
D478 9
D551 9
D553 9
D607 9
D608 9
D703 9
361277
(20634, 8)


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

361278
(10000, 22)


In [2]:
import os
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import lightgbmlss
import optuna
from scipy.spatial.distance import mahalanobis
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from properscoring import crps_gaussian, crps_ensemble
import random
import gpytorch
import tqdm.auto as tqdm
from lightgbmlss.model import *
from lightgbmlss.distributions.Gaussian import *
from drf import drf
from pygam import LinearGAM, s, f
from utils import EarlyStopping, train, train_trans, train_no_early_stopping, train_trans_no_early_stopping, train_GP, ExactGPModel
from torch.utils.data import TensorDataset, DataLoader

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task_id=361072

print(f"Task {task_id}")

# Create the checkpoint directory if it doesn't exist
os.makedirs('CHECKPOINTS/SPATIAL_DEPTH', exist_ok=True)
CHECKPOINT_PATH = f'CHECKPOINTS/SPATIAL_DEPTH/task_{task_id}.pt'

print(f"Task {task_id}")

task = openml.tasks.get_task(task_id)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

# Find features with absolute correlation > 0.9
corr_matrix = X.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

# Drop one of the highly correlated features
X = X.drop(high_corr_features, axis=1)

# Set the random seed for reproducibility
N_TRIALS=100
N_SAMPLES=100
PATIENCE=40
N_EPOCHS=1000
GP_ITERATIONS=1000
BATCH_SIZE=1024
seed=10
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)


# activate pandas conversion for rpy2
pandas2ri.activate()

# import R's "ddalpha" package
ddalpha = importr('ddalpha')

# explicitly import the projDepth function
spatialDepth = robjects.r['depth.spatial']

# calculate the spatial depth for each data point
spatial_depth = spatialDepth(X, X)

spatial_depth=pd.Series(spatial_depth,index=X.index)
far_index=spatial_depth.index[np.where(spatial_depth<=np.quantile(spatial_depth,0.2))[0]]
close_index=spatial_depth.index[np.where(spatial_depth>np.quantile(spatial_depth,0.2))[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

# convert the R vector to a pandas Series
spatial_depth_ = spatialDepth(X_train, X_train)

spatial_depth_=pd.Series(spatial_depth_,index=X_train.index)
far_index_=spatial_depth_.index[np.where(spatial_depth_<=np.quantile(spatial_depth_,0.2))[0]]
close_index_=spatial_depth_.index[np.where(spatial_depth_>np.quantile(spatial_depth_,0.2))[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]


# Standardize the data
mean_X_train_ = np.mean(X_train_, axis=0)
std_X_train_ = np.std(X_train_, axis=0)
X_train__scaled = (X_train_ - mean_X_train_) / std_X_train_
X_val_scaled = (X_val - mean_X_train_) / std_X_train_

mean_X_train = np.mean(X_train, axis=0)
std_X_train = np.std(X_train, axis=0)
X_train_scaled = (X_train - mean_X_train) / std_X_train
X_test_scaled = (X_test - mean_X_train) / std_X_train


# Convert data to PyTorch tensors
X_train__tensor = torch.tensor(X_train__scaled.values, dtype=torch.float32)
y_train__tensor = torch.tensor(y_train_.values, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert to use GPU if available
if torch.cuda.is_available():
    X_train__tensor = X_train__tensor.cuda()
    y_train__tensor = y_train__tensor.cuda()
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

# Create flattened versions of the data
y_val_np = y_val.values.flatten()
y_test_np = y_test.values.flatten()

# Create TensorDatasets for training and validation sets
train__dataset = TensorDataset(X_train__tensor, y_train__tensor)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for training and validation sets
train__loader = DataLoader(train__dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define d_out and d_in
d_out = 1  
d_in=X_train_.shape[1]

Task 361072
Task 361072


Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.


In [8]:
constant_prediction_train = np.array([np.mean(y_train)]*len(y_train))
# Calculate the standard deviation of the residuals
std_dev = np.std(y_train - constant_prediction_train)
constant_prediction = np.array([np.mean(y_train)]*len(y_test))
# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=constant_prediction[i], sig=std_dev) for i in range(len(y_test_np))]
CRPS_constant = np.mean(crps_values)
CRPS_constant

21.228778214732248

In [9]:
constant_prediction

array([88.82481306, 88.82481306, 88.82481306, ..., 88.82481306,
       88.82481306, 88.82481306])

In [7]:
CRPS_constant

148.67837458822024

[26.782906346427744,
 185.55074938195384,
 26.79682991991504,
 110.59774647665711,
 26.814233393032794,
 187.49243491262348,
 26.838596400639656,
 177.8113249737905,
 157.74788152939988,
 180.70825991211495,
 110.59774647665711,
 180.70825991211495,
 180.70825991211495,
 26.782906346427744,
 183.61169292788176,
 185.55074938195384,
 26.782906346427744,
 110.59774647665711,
 184.58088626734454,
 184.58088626734454,
 176.84718750657188,
 26.869917088485956,
 167.25097941447643,
 168.20667799635712,
 189.43665132362818,
 110.59774647665711,
 110.59774647665711,
 110.59774647665711,
 173.95949504068867,
 177.8113249737905,
 183.61169292788176,
 26.786387306053033,
 110.59774647665711,
 176.84718750657188,
 110.59774647665711,
 177.8113249737905,
 110.59774647665711,
 183.61169292788176,
 165.34238663820142,
 110.59774647665711,
 173.95949504068867,
 184.58088626734454,
 26.782906346427744,
 110.59774647665711,
 26.79682991991504,
 162.4867573925009,
 26.838596400639656,
 174.92125813966973

In [4]:
constant_prediction

array([88, 88, 88, ..., 88, 88, 88], dtype=uint8)

In [5]:
y_train

0       90
2       85
4       79
5       92
6       82
        ..
8186    85
8188    88
8189    92
8190    96
8191    80
Name: usr, Length: 6553, dtype: uint8

In [3]:
#SUITE_ID = 336 # Regression on numerical features
SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite
print(len(benchmark_suite.tasks))

for task_id in benchmark_suite.tasks:
    print(task_id)
    task = openml.tasks.get_task(task_id)  # download the OpenML task
    dataset = task.get_dataset()

    X, y, categorical_indicator, attribute_names = dataset.get_data(
            dataset_format="dataframe", target=dataset.default_target_attribute)
    print(X.shape)

16
361055


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


(16714, 10)
361060
(38474, 7)
361061


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

(566602, 10)
361062
(10082, 26)
361063
(13488, 16)
361065


...

(57580, 54)
361275
(13272, 20)
361276


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

(3434, 419)
361277
(20634, 8)
361278


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will defa

(10000, 22)


In [1]:
#SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite
print(len(benchmark_suite.tasks))

for task_id in benchmark_suite.tasks:
    print(task_id)
    task = openml.tasks.get_task(task_id)  # download the OpenML task
    dataset = task.get_dataset()

    X, y, categorical_indicator, attribute_names = dataset.get_data(
            dataset_format="dataframe", target=dataset.default_target_attribute)
    print(X.shape)
    print(X.value_counts)

NameError: name 'openml' is not defined

In [20]:
benchmark_suite.tasks

[361072,
 361073,
 361074,
 361076,
 361077,
 361078,
 361079,
 361080,
 361081,
 361082,
 361083,
 361084,
 361085,
 361086,
 361087,
 361088,
 361279,
 361280,
 361281]

In [8]:
print(X.shape[1])

16


In [4]:
task_id=361074

print(f"Task {task_id}")

# Create the checkpoint directory if it doesn't exist
os.makedirs('CHECKPOINTS/MAHALANOBIS', exist_ok=True)
CHECKPOINT_PATH = f'CHECKPOINTS/MAHALANOBIS/task_{task_id}.pt'

print(f"Task {task_id}")

task = openml.tasks.get_task(task_id)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)


Task 361074
Task 361074


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [22]:
X

,climbRate,Sgz,p,q,curRoll,absRoll,diffClb,diffRollRate,diffDiffClb,SaTime1,SaTime2,SaTime3,SaTime4,diffSaTime1,diffSaTime3,Sa
0,118.0,-55.0,-0.28,-0.08,-0.2,-11.0,11.0,0.005,-0.2,-0.0010,-0.0010,-0.0010,-0.0010,0.0000,0.0,-0.0010
1,390.0,-45.0,-0.06,-0.07,-0.6,-12.0,11.0,0.010,-0.2,-0.0008,-0.0008,-0.0008,-0.0008,0.0000,0.0,-0.0008
2,68.0,6.0,0.11,0.15,0.6,-10.0,-9.0,-0.003,-0.2,-0.0011,-0.0010,-0.0010,-0.0010,-0.0002,0.0,-0.0010
3,-358.0,-12.0,-0.20,0.13,-0.3,-11.0,-7.0,0.001,-0.1,-0.0010,-0.0010,-0.0010,-0.0010,0.0000,0.0,-0.0010
4,-411.0,-19.0,-0.18,0.02,-0.5,-11.0,-3.0,0.002,1.2,-0.0010,-0.0010,-0.0010,-0.0010,0.0000,0.0,-0.0010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16594,299.0,-28.0,0.08,-0.12,-0.3,-9.0,15.0,0.010,-0.2,-0.0005,-0.0005,-0.0005,-0.0005,0.0000,0.0,-0.0005
16595,84.0,0.0,0.14,0.14,1.1,-8.0,-11.0,-0.014,-0.6,-0.0009,-0.0009,-0.0009,-0.0009,0.0000,0.0,-0.0009
16596,-208.0,-6.0,-0.48,0.09,0.2,-9.0,-7.0,-0.010,-0.1,-0.0009,-0.0009,-0.0009,-0.0009,0.0000,0.0,-0.0009
16597,-146.0,-14.0,-0.38,-0.03,-0.8,-10.0,10.0,0.010,-1.0,-0.0005,-0.0005,-0.0005,-0.0005,0.0000,0.0,-0.0005


In [12]:
#SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite
print(len(benchmark_suite.tasks))

7


In [6]:
for i in range(5):
    if i==2:
        continue
    print(i)

0
1
3
4


In [7]:
len(X)

10000

In [18]:
from umap import UMAP
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import optuna
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from properscoring import crps_gaussian, crps_ensemble
import random
import gpytorch
import tqdm.auto as tqdm
from sklearn.metrics.pairwise import euclidean_distances
import os
from pygam import LinearGAM, s, f
from utils import EarlyStopping, train, train_trans, train_no_early_stopping, train_trans_no_early_stopping, train_GP, ExactGPModel
from torch.utils.data import TensorDataset, DataLoader
import re
import shutil

# Create the checkpoint directory if it doesn't exist
if os.path.exists('CHECKPOINTS/MAHALANOBIS'):
    shutil.rmtree('CHECKPOINTS/MAHALANOBIS')
os.makedirs('CHECKPOINTS/MAHALANOBIS')

SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task_id=361072

# Set the random seed for reproducibility
N_TRIALS=100
N_SAMPLES=100
PATIENCE=40
N_EPOCHS=1000
GP_ITERATIONS=1000
BATCH_SIZE=1024
seed=10
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)

print(f"Task {task_id}")

CHECKPOINT_PATH = f'CHECKPOINTS/MAHALANOBIS/task_{task_id}.pt'

task = openml.tasks.get_task(task_id)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

if len(X) > 15000:
    indices = np.random.choice(X.index, size=15000, replace=False)
    X = X.iloc[indices,]
    y = y[indices]

# Remove categorical columns with more than 20 unique values and non-categorical columns with less than 10 unique values
# Remove non-categorical columns with more than 70% of the data in one category from X_clean
for col in [attribute for attribute, indicator in zip(attribute_names, categorical_indicator) if indicator]:
    if len(X[col].unique()) > 20:
        X = X.drop(col, axis=1)

X_clean=X.copy()
for col in [attribute for attribute, indicator in zip(attribute_names, categorical_indicator) if not indicator]:
    if len(X[col].unique()) < 10:
        X = X.drop(col, axis=1)
        X_clean = X_clean.drop(col, axis=1)
    elif X[col].value_counts(normalize=True).max() > 0.7:
        X_clean = X_clean.drop(col, axis=1)

# Find features with absolute correlation > 0.9
corr_matrix = X_clean.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

# Drop one of the highly correlated features from X_clean
X_clean = X_clean.drop(high_corr_features, axis=1)

# Rename columns to avoid problems with LGBM
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# Apply UMAP decomposition
umap = UMAP(n_components=2, random_state=42)
X_umap = umap.fit_transform(X_clean)

# calculate the Euclidean distance matrix
euclidean_dist_matrix = euclidean_distances(X_umap)

# calculate the Euclidean distance for each data point
euclidean_dist = np.mean(euclidean_dist_matrix, axis=1)

euclidean_dist = pd.Series(euclidean_dist, index=X_clean.index)
far_index = euclidean_dist.index[np.where(euclidean_dist >= np.quantile(euclidean_dist, 0.8))[0]]
close_index = euclidean_dist.index[np.where(euclidean_dist < np.quantile(euclidean_dist, 0.8))[0]]

X_train_clean = X_clean.loc[close_index,:]
X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

# Apply UMAP decomposition on the training set
X_umap_train = umap.fit_transform(X_train_clean)

# calculate the Euclidean distance matrix for the training set
euclidean_dist_matrix_train = euclidean_distances(X_umap_train)

# calculate the Euclidean distance for each data point in the training set
euclidean_dist_train = np.mean(euclidean_dist_matrix_train, axis=1)

euclidean_dist_train = pd.Series(euclidean_dist_train, index=X_train_clean.index)
far_index_train = euclidean_dist_train.index[np.where(euclidean_dist_train >= np.quantile(euclidean_dist_train, 0.8))[0]]
close_index_train = euclidean_dist_train.index[np.where(euclidean_dist_train < np.quantile(euclidean_dist_train, 0.8))[0]]

X_train_ = X_train.loc[close_index_train,:]
X_val = X_train.loc[far_index_train,:]
y_train_ = y_train.loc[close_index_train]
y_val = y_train.loc[far_index_train]

# Standardize the data
mean_X_train_ = np.mean(X_train_, axis=0)
std_X_train_ = np.std(X_train_, axis=0)
X_train__scaled = (X_train_ - mean_X_train_) / std_X_train_
X_val_scaled = (X_val - mean_X_train_) / std_X_train_

mean_X_train = np.mean(X_train, axis=0)
std_X_train = np.std(X_train, axis=0)
X_train_scaled = (X_train - mean_X_train) / std_X_train
X_test_scaled = (X_test - mean_X_train) / std_X_train


# Convert data to PyTorch tensors
X_train__tensor = torch.tensor(X_train__scaled.values, dtype=torch.float32)
y_train__tensor = torch.tensor(y_train_.values, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_scaled.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert to use GPU if available
if torch.cuda.is_available():
    print("Using GPU")
    X_train__tensor = X_train__tensor.cuda()
    y_train__tensor = y_train__tensor.cuda()
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()
else:
    print("Using CPU")

# Create flattened versions of the data
y_val_np = y_val.values.flatten()
y_test_np = y_test.values.flatten()

# Create TensorDatasets for training and validation sets
train__dataset = TensorDataset(X_train__tensor, y_train__tensor)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders for training and validation sets
train__loader = DataLoader(train__dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define d_out and d_in
d_out = 1  
d_in=X_train_.shape[1]

Task 361072


c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed 

Using CPU


In [55]:
N_TRIALS=2
#### GAM model
def gam_model(trial):

    n_splines = []
    lam = []
    spline_order = []

    # Iterate over each covariate in X_train_
    for col in X_train_.columns:
        # Define the search space for n_splines, lam, and spline_order
        n_splines.append(trial.suggest_int(f'n_splines_{col}', 10, 100))
        lam.append(trial.suggest_float(f'lam_{col}', 1e-3, 1e3, log=True))
        spline_order.append(trial.suggest_int(f'spline_order_{col}', 1, 5))
    
    # Create and train the model
    gam = LinearGAM(n_splines=n_splines, spline_order=spline_order, lam=lam).fit(X_train_, y_train_)

    # Predict on the validation set and calculate the CRPS
    y_train__hat_gam = gam.predict(X_train_)
    std_dev_error = np.std(y_train_ - y_train__hat_gam)
    y_val_hat_gam = gam.predict(X_val)
    crps_gam = [crps_gaussian(y_val_np[i], mu=y_val_hat_gam[i], sig=std_dev_error) for i in range(len(y_val_hat_gam))]
    crps_gam = np.mean(crps_gam)

    return crps_gam

# Create the sampler and study
sampler_gam = optuna.samplers.TPESampler(seed=seed)
study_gam = optuna.create_study(sampler=sampler_gam, direction='minimize')

# Optimize the model
study_gam.optimize(gam_model, n_trials=N_TRIALS)

n_splines = []
lam = []
spline_order = []

# Create the final model with the best parameters
best_params = study_gam.best_params

# Iterate over each covariate in X_train_
for col in X_train.columns:
    # Define the search space for n_splines, lam, and spline_order
    n_splines.append(best_params[f'n_splines_{col}'])
    lam.append(best_params[f'lam_{col}'])
    spline_order.append(best_params[f'spline_order_{col}'])

final_gam_model = LinearGAM(n_splines=n_splines, spline_order=spline_order, lam=lam)

# Fit the model
final_gam_model.fit(X_train, y_train)

# Predict on the train set
y_train_hat_gam = final_gam_model.predict(X_train)
std_dev_error = np.std(y_train - y_train_hat_gam)

# Predict on the test set
y_test_hat_gam = final_gam_model.predict(X_test)

# Calculate the CRPS
crps_gam = [crps_gaussian(y_test_np[i], mu=y_test_hat_gam[i], sig=std_dev_error) for i in range(len(y_test_hat_gam))]
crps_gam = np.mean(crps_gam)
print("CRPS GAM: ", crps_gam)

[I 2024-03-27 21:12:39,629] A new study created in memory with name: no-name-e8758b3d-c272-427a-a523-34dc4e18a9c7


[I 2024-03-27 21:12:48,947] Trial 0 finished with value: 2.136980892037132 and parameters: {'n_splines_lread': 80, 'lam_lread': 0.0013320229150659071, 'spline_order_lread': 4, 'n_splines_lwrite': 78, 'lam_lwrite': 0.9795848815655198, 'spline_order_lwrite': 2, 'n_splines_scall': 28, 'lam_scall': 36.57499481192727, 'spline_order_scall': 1, 'n_splines_sread': 18, 'lam_sread': 12.94669479886693, 'spline_order_sread': 5, 'n_splines_swrite': 10, 'lam_swrite': 1.1834599907542849, 'spline_order_swrite': 5, 'n_splines_fork': 65, 'lam_fork': 21.405821746591823, 'spline_order_fork': 2, 'n_splines_exec': 93, 'lam_exec': 19.384504329781148, 'spline_order_exec': 3, 'n_splines_rchar': 22, 'lam_rchar': 0.173797914304444, 'spline_order_rchar': 4, 'n_splines_wchar': 50, 'lam_wchar': 0.4018684945839337, 'spline_order_wchar': 4, 'n_splines_pgout': 56, 'lam_pgout': 7.986989097169455, 'spline_order_pgout': 4, 'n_splines_ppgout': 83, 'lam_ppgout': 1.3485992516968202, 'spline_order_ppgout': 5, 'n_splines_pgfr

CRPS GAM:  12.33195379119565


In [58]:
#### GAM model
N_TRIALS=100
def gam_model(trial):

    n_splines = []
    lam = []
    spline_order = []

    # Iterate over each covariate in X_train_
    for col in X_train_.columns:
        # Define the search space for n_splines, lam, and spline_order
        n_splines.append(trial.suggest_int(f'n_splines_{col}', 10, 100))
        lam.append(trial.suggest_float(f'lam_{col}', 1e-3, 1e3, log=True))
        spline_order.append(trial.suggest_int(f'spline_order_{col}', 1, 5))
    
    # Create and train the model
    gam = LinearGAM(n_splines=n_splines, spline_order=spline_order, lam=lam).fit(X_train_, y_train_)

    # Predict on the validation set and calculate the RMSE
    y_val_hat_gam = gam.predict(X_val)
    RMSE_gam = np.sqrt(np.mean((y_val - y_val_hat_gam) ** 2))

    return RMSE_gam

# Create the sampler and study
sampler_gam = optuna.samplers.TPESampler(seed=seed)
study_gam = optuna.create_study(sampler=sampler_gam, direction='minimize')

# Optimize the model
study_gam.optimize(gam_model, n_trials=N_TRIALS)

n_splines = []
lam = []
spline_order = []

# Create the final model with the best parameters
best_params = study_gam.best_params

# Iterate over each covariate in X_train_
for col in X_train.columns:
    # Define the search space for n_splines, lam, and spline_order
    n_splines.append(best_params[f'n_splines_{col}'])
    lam.append(best_params[f'lam_{col}'])
    spline_order.append(best_params[f'spline_order_{col}'])

final_gam_model = LinearGAM(n_splines=n_splines, spline_order=spline_order, lam=lam)

# Fit the model
final_gam_model.fit(X_train, y_train)

# Predict on the test set
y_test_hat_gam = final_gam_model.predict(X_test)
# Calculate the RMSE
RMSE_gam = np.sqrt(np.mean((y_test - y_test_hat_gam) ** 2))
print("RMSE GAM: ", RMSE_gam)

[I 2024-03-27 21:25:27,265] A new study created in memory with name: no-name-24ab0b40-f220-45a3-aa79-ea5da9acd6b5
[I 2024-03-27 21:25:37,337] Trial 0 finished with value: 14.062515995017966 and parameters: {'n_splines_lread': 80, 'lam_lread': 0.0013320229150659071, 'spline_order_lread': 4, 'n_splines_lwrite': 78, 'lam_lwrite': 0.9795848815655198, 'spline_order_lwrite': 2, 'n_splines_scall': 28, 'lam_scall': 36.57499481192727, 'spline_order_scall': 1, 'n_splines_sread': 18, 'lam_sread': 12.94669479886693, 'spline_order_sread': 5, 'n_splines_swrite': 10, 'lam_swrite': 1.1834599907542849, 'spline_order_swrite': 5, 'n_splines_fork': 65, 'lam_fork': 21.405821746591823, 'spline_order_fork': 2, 'n_splines_exec': 93, 'lam_exec': 19.384504329781148, 'spline_order_exec': 3, 'n_splines_rchar': 22, 'lam_rchar': 0.173797914304444, 'spline_order_rchar': 4, 'n_splines_wchar': 50, 'lam_wchar': 0.4018684945839337, 'spline_order_wchar': 4, 'n_splines_pgout': 56, 'lam_pgout': 7.986989097169455, 'spline_o

RMSE GAM:  40.98465473045525


In [60]:
#### GAM model
N_TRIALS=100
def gam_model(trial):

    # Define the search space for n_splines, lam, and spline_order
    params={'n_splines': trial.suggest_int('n_splines', 10, 100),
    'lam':trial.suggest_float('lam', 1e-3, 1e3, log=True),
    'spline_order':trial.suggest_int('spline_order', 1, 5)}
    
    # Create and train the model
    gam = LinearGAM(n_splines=params['n_splines'], spline_order=params['spline_order'], lam=params['lam']).fit(X_train_, y_train_)

    # Predict on the validation set and calculate the RMSE
    y_val_hat_gam = gam.predict(X_val)
    RMSE_gam = np.sqrt(np.mean((y_val - y_val_hat_gam) ** 2))

    return RMSE_gam

# Create the sampler and study
sampler_gam = optuna.samplers.TPESampler(seed=seed)
study_gam = optuna.create_study(sampler=sampler_gam, direction='minimize')

# Optimize the model
study_gam.optimize(gam_model, n_trials=N_TRIALS)

# Create the final model with the best parameters
best_params = study_gam.best_params

final_gam_model = LinearGAM(n_splines=best_params['n_splines'], spline_order=best_params['spline_order'], lam=best_params['lam'])

# Fit the model
final_gam_model.fit(X_train, y_train)

# Predict on the test set
y_test_hat_gam = final_gam_model.predict(X_test)
# Calculate the RMSE
RMSE_gam = np.sqrt(np.mean((y_test - y_test_hat_gam) ** 2))
print("RMSE GAM: ", RMSE_gam)

[I 2024-03-27 21:55:36,796] A new study created in memory with name: no-name-968777f5-72dc-47ba-b1ec-f8b3a77ff081
[I 2024-03-27 21:56:01,184] Trial 0 finished with value: 334.1580013815291 and parameters: {'n_splines': 80, 'lam': 0.0013320229150659071, 'spline_order': 4}. Best is trial 0 with value: 334.1580013815291.
[I 2024-03-27 21:56:24,102] Trial 1 finished with value: 113.54695068295823 and parameters: {'n_splines': 78, 'lam': 0.9795848815655198, 'spline_order': 2}. Best is trial 1 with value: 113.54695068295823.
[I 2024-03-27 21:56:27,245] Trial 2 finished with value: 20.56405329642756 and parameters: {'n_splines': 28, 'lam': 36.57499481192727, 'spline_order': 1}. Best is trial 2 with value: 20.56405329642756.
[I 2024-03-27 21:56:29,219] Trial 3 finished with value: 13.648983422833867 and parameters: {'n_splines': 18, 'lam': 12.94669479886693, 'spline_order': 5}. Best is trial 3 with value: 13.648983422833867.
[I 2024-03-27 21:56:30,430] Trial 4 finished with value: 9.7905105625

RMSE GAM:  33.18405963367804


In [44]:
# Create the final model with the best parameters
best_params = study_gam.best_params

In [46]:
n_splines = []
lam = []
spline_order = []

# Iterate over each covariate in X_train_
for col in X_train.columns:
    # Define the search space for n_splines, lam, and spline_order
    n_splines.append(best_params[f'n_splines_{col}'])
    lam.append(best_params[f'lam_{col}'])
    spline_order.append(best_params[f'spline_order_{col}'])

final_gam_model = LinearGAM(n_splines=n_splines, spline_order=spline_order, lam=lam)

# Fit the model
final_gam_model.fit(X_train, y_train)

# Predict on the train set
y_train_hat_gam = final_gam_model.predict(X_train)
std_dev_error = np.std(y_train - y_train_hat_gam)

# Predict on the test set
y_test_hat_gam = final_gam_model.predict(X_test)

# Calculate the CRPS
crps_gam = [crps_gaussian(y_test_np[i], mu=y_test_hat_gam[i], sig=std_dev_error) for i in range(len(y_test_hat_gam))]
crps_gam = np.mean(crps_gam)
print("CRPS GAM: ", crps_gam)

In [39]:
N_TRIALS=2
#### GAM model
def gam_model(trial):

    # Define the hyperparameters to optimize
    params = {'n_splines': trial.suggest_int('n_splines', 10, 100),
              'lam': trial.suggest_float('lam', 1e-3, 1e3, log=True),
              'spline_order': trial.suggest_int('spline_order', 1, 5)}
    
    # Create and train the model
    gam = LinearGAM(n_splines=params['n_splines'], spline_order=params['spline_order'], lam=params['lam']).fit(X_train_, y_train_)

    # Predict on the validation set and calculate the CRPS
    y_train_hat_gam = gam.predict(X_train)
    std_dev_error = np.std(y_train - y_train_hat_gam)
    y_val_hat_gam = gam.predict(X_val)
    crps_gam = [crps_gaussian(y_val_np[i], mu=y_val_hat_gam[i], sig=std_dev_error) for i in range(len(y_val_hat_gam))]
    crps_gam = np.mean(crps_gam)

    return crps_gam

# Create the sampler and study
sampler_gam = optuna.samplers.TPESampler(seed=seed)
study_gam = optuna.create_study(sampler=sampler_gam, direction='minimize')

# Optimize the model
study_gam.optimize(gam_model, n_trials=N_TRIALS)

# Create the final model with the best parameters
best_params = study_gam.best_params
final_gam_model = LinearGAM(n_splines=best_params['n_splines'], spline_order=best_params['spline_order'], lam=best_params['lam'])

# Fit the model
final_gam_model.fit(X_train, y_train)

# Predict on the train set
y_train_hat_gam = final_gam_model.predict(X_train)
std_dev_error = np.std(y_train - y_train_hat_gam)

# Predict on the test set
y_test_hat_gam = final_gam_model.predict(X_test)

# Calculate the CRPS
crps_gam = [crps_gaussian(y_test_np[i], mu=y_test_hat_gam[i], sig=std_dev_error) for i in range(len(y_test_hat_gam))]
crps_gam = np.mean(crps_gam)
print("CRPS GAM: ", crps_gam)

[I 2024-03-27 20:45:26,719] A new study created in memory with name: no-name-f0eb0079-46e2-4e12-9380-3ad1264ea862
[I 2024-03-27 20:45:54,208] Trial 0 finished with value: 38.545020865511354 and parameters: {'n_splines': 79, 'lam': 0.0013320229150659071, 'spline_order': 4}. Best is trial 0 with value: 38.545020865511354.
[I 2024-03-27 20:46:16,055] Trial 1 finished with value: 13.423005516937682 and parameters: {'n_splines': 76, 'lam': 0.9795848815655198, 'spline_order': 2}. Best is trial 1 with value: 13.423005516937682.


CRPS GAM:  5.158456944302808


In [53]:
final_gam_model.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                    483.5689
Link Function:                     IdentityLink Log Likelihood:                                -17415.2801
Number of Samples:                         6553 AIC:                                             35799.698
                                                AICc:                                           35877.2568
                                                GCV:                                                5.9962
                                                Scale:                                              5.2054
                                                Pseudo R-Squared:                                   0.9394
Feature Function                  Lam

C:\Users\dalma\AppData\Local\Temp\ipykernel_2548\3018071388.py:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  final_gam_model.summary()


In [27]:
# Create and train the model
gam = LinearGAM(s(0, n_splines=20, spline_order=5)).gridsearch(X_train_, y_train_, lam=np.logspace(-3, 3, 11))

InvalidIndexError: (slice(None, None, None), 0)

In [13]:
X_train

,lread,lwrite,scall,sread,swrite,fork,exec,rchar,wchar,pgout,...,pgfree,pgscan,atch,pgin,ppgin,pflt,vflt,runqsz,freemem,freeswap
0,6.0,2.0,1036.0,103.0,114.0,1.00,1.00,172076.0,355965.0,0.00,...,0.00,0.00,0.00,2.00,4.00,73.60,89.00,2.0,6527.0,1851864.0
2,62.0,77.0,3806.0,258.0,166.0,1.40,1.40,492142.0,268706.0,4.80,...,44.00,79.20,2.20,7.60,12.20,68.00,218.80,5.2,256.0,1314590.0
3,5.0,0.0,4721.0,256.0,177.0,0.99,2.58,524787.0,174964.0,14.51,...,88.47,189.86,1.99,4.17,24.85,95.63,248.91,1.0,233.0,972606.0
5,5.0,1.0,1692.0,132.0,87.0,0.40,1.80,220194.0,107031.0,0.00,...,0.00,0.00,0.00,3.81,4.01,23.65,42.08,2.2,2291.0,1010703.0
6,3.0,0.0,635.0,65.0,47.0,3.00,3.00,87465.0,40740.0,8.60,...,8.80,0.00,0.00,1.60,2.20,227.60,226.40,1.0,289.0,1806587.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8186,15.0,11.0,2599.0,277.0,234.0,3.61,2.20,133465.0,102035.0,17.03,...,50.10,46.29,10.22,10.02,15.83,187.78,297.80,2.0,139.0,1731732.0
8188,29.0,40.0,1906.0,118.0,90.0,0.80,2.00,8175.0,27313.0,0.00,...,0.00,0.00,0.00,0.80,0.80,56.20,78.60,3.6,166.0,1107088.0
8189,3.0,0.0,926.0,90.0,67.0,0.60,1.00,5411.0,19322.0,0.00,...,0.00,0.00,0.40,0.40,0.40,53.40,154.00,1.0,1177.0,1020400.0
8190,4.0,0.0,418.0,30.0,29.0,0.80,1.00,3959.0,10679.0,0.00,...,0.00,0.00,0.00,0.20,0.20,61.00,73.20,2.4,6355.0,1702592.0


In [14]:
X_test

,lread,lwrite,scall,sread,swrite,fork,exec,rchar,wchar,pgout,...,pgfree,pgscan,atch,pgin,ppgin,pflt,vflt,runqsz,freemem,freeswap
1,1.0,0.0,2165.0,205.0,101.0,0.4,1.20,43107.0,44139.0,4.8,...,75.8,181.4,0.2,85.40,88.20,19.40,161.8,3.0,130.0,1131931.0
4,42.0,55.0,3949.0,249.0,244.0,2.6,4.60,197289.0,529200.0,4.2,...,6.6,0.0,1.4,1.80,2.20,219.60,297.2,3.4,331.0,1013805.0
7,7.0,5.0,1341.0,240.0,120.0,0.4,0.60,718437.0,672290.0,0.0,...,0.0,0.0,0.0,7.60,15.00,42.80,52.8,4.8,2532.0,1037078.0
14,172.0,113.0,1427.0,171.0,134.0,0.6,3.39,261636.0,103649.0,0.0,...,0.0,0.0,0.8,4.99,5.39,62.28,152.1,744.0,92.0,8.0
16,37.0,49.0,4667.0,673.0,457.0,2.6,0.80,262181.0,220110.0,0.2,...,6.6,9.2,0.2,3.40,3.40,130.80,212.8,1.4,287.0,1073834.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8172,4.0,0.0,2075.0,254.0,204.0,3.4,1.00,110184.0,69205.0,7.8,...,59.6,96.2,1.6,82.00,82.40,164.60,374.0,1.4,130.0,1106848.0
8176,71.0,80.0,1052.0,221.0,157.0,0.4,0.40,145004.0,48770.0,1.4,...,3.8,3.8,3.6,10.40,15.40,36.60,120.4,896.0,89.0,11.0
8177,40.0,1.0,2330.0,228.0,81.0,10.6,21.80,955802.0,67045.0,7.0,...,68.6,101.2,1.4,51.80,53.20,359.60,761.8,4.4,122.0,1107003.0
8181,10.0,2.0,2981.0,190.0,107.0,1.0,2.00,670209.0,86240.0,16.0,...,233.4,460.0,0.4,62.60,108.80,75.00,367.8,5.6,137.0,1317451.0


In [7]:
X

,lread,lwrite,scall,sread,swrite,fork,exec,rchar,wchar,pgout,ppgout,pgfree,atch,pgin,ppgin,pflt,vflt,runqsz,freemem,freeswap
lread,1.000000,0.533737,0.191377,0.132881,0.119953,0.140284,0.110965,0.109624,0.081270,0.082463,0.130590,0.114438,0.021563,0.189799,0.161345,0.137463,0.165539,0.023379,-0.083214,-0.081293
lwrite,0.533737,1.000000,0.143404,0.128403,0.101524,0.052511,0.038237,0.117464,0.091319,0.067013,0.079485,0.065692,0.028310,0.091068,0.089011,0.067024,0.094965,0.043562,-0.091133,-0.116478
scall,0.191377,0.143404,1.000000,0.696887,0.619984,0.446766,0.308999,0.352609,0.274199,0.194529,0.208400,0.199778,0.077969,0.241628,0.219070,0.481781,0.531760,0.000162,-0.387520,-0.350629
sread,0.132881,0.128403,0.696887,1.000000,0.881069,0.416721,0.164084,0.504412,0.401923,0.193679,0.225906,0.212911,0.085468,0.207012,0.210225,0.452020,0.491045,0.041096,-0.286437,-0.302036
swrite,0.119953,0.101524,0.619984,0.881069,1.000000,0.376876,0.103643,0.333322,0.394356,0.151371,0.159442,0.145458,0.061373,0.147000,0.144278,0.396580,0.416571,0.014101,-0.248574,-0.237062
fork,0.140284,0.052511,0.446766,0.416721,0.376876,1.000000,0.763974,0.280955,0.061040,0.130192,0.166872,0.168082,0.047194,0.163468,0.132181,0.931040,0.939348,-0.017393,-0.123357,-0.130442
exec,0.110965,0.038237,0.308999,0.164084,0.103643,0.763974,1.000000,0.168085,0.000640,0.111465,0.149334,0.146163,0.052307,0.186099,0.149911,0.645239,0.691754,-0.006048,-0.158565,-0.153347
rchar,0.109624,0.117464,0.352609,0.504412,0.333322,0.280955,0.168085,1.000000,0.505186,0.210885,0.268056,0.276503,0.171322,0.299051,0.346427,0.313150,0.363618,0.131645,-0.149771,-0.220934
wchar,0.081270,0.091319,0.274199,0.401923,0.394356,0.061040,0.000640,0.505186,1.000000,0.192523,0.188655,0.159009,0.181452,0.177709,0.200614,0.086396,0.111358,0.191111,-0.149335,-0.225983
pgout,0.082463,0.067013,0.194529,0.193679,0.151371,0.130192,0.111465,0.210885,0.192523,1.000000,0.872445,0.730381,0.147759,0.385648,0.414865,0.151285,0.229129,-0.009814,-0.269687,-0.245378
